# Coursework 2



In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt

The `monte_carlo(μ, σ, price, days, iterations, rng)` function below simulates a share price defined by the equation $\frac{P_t}{P_{t-1}}=Z$ where $P_t$ is the price at time $t$ and $Z$ is a random draw from a normal distribution with a mean passed as $\mu$ and a standard deviation passed as $\sigma$.
_price_ is the initial price of the share on day 1, _days_ is the number of days to run the simulation for, _iterations_ is the number of times to repeat the simulation, and _rng_ is a [`numpy.random.Generator`](https://numpy.org/doc/stable/reference/random/generator) to supply the random numbers (e.g. if you need a fixed seed).

In [ ]:
def monte_carlo(μ=0, σ=100, price=100, days=252, iterations=1, rng=None):
    μ = np.atleast_1d(μ)/100
    σ = np.atleast_1d(σ)/100
    variations = max(μ.size, σ.size)
    if rng is None:
        rng = np.random.default_rng()
    shape = (days, iterations, variations)
    Z = np.zeros(shape)
    for i in range(variations):
        Z[...,i] = rng.normal(loc=1+μ[i%μ.size], scale=σ[i%σ.size], size=shape[0:2])
    P = np.empty(shape)
    P[0,...] = price
    for i in range(1, days):
        P[i,...] = P[i-1,...] * Z[i,...]
    return P